In [1]:
!pip install kaggle tensorflow

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [10]:
#DATA COLLECTION- KAGGLE API
kaggle_dictionary = json.load(open(r"C:\Users\nehaa\Downloads\kaggle.json"))

In [11]:
kaggle_dictionary.keys()

dict_keys(['username', 'key'])

In [12]:
# setup kaggle credentials as environment variables
os.environ["KAGGLE_USERNAME"] = kaggle_dictionary["username"]
os.environ["KAGGLE_KEY"] = kaggle_dictionary["key"]

In [13]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [14]:
# unzip the dataset file
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip", "r") as zip_ref:
  zip_ref.extractall()

In [15]:
#Loading the dataset
data = pd.read_csv("IMDB Dataset.csv")

In [16]:
data.shape

(50000, 2)

In [17]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [18]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [19]:
data["sentiment"].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [20]:
import pandas as pd

# Set the pandas option to opt-in to the future behavior
pd.set_option('future.no_silent_downcasting', True)

# Sample DataFrame
'''data = pd.DataFrame({
    'sentiment': ['positive', 'negative', 'positive', 'negative']
})'''

# Replace values
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

print(data)


                                                  review sentiment
0      One of the other reviewers has mentioned that ...         1
1      A wonderful little production. <br /><br />The...         1
2      I thought this was a wonderful way to spend ti...         1
3      Basically there's a family where a little boy ...         0
4      Petter Mattei's "Love in the Time of Money" is...         1
...                                                  ...       ...
49995  I thought this movie did a down right good job...         1
49996  Bad plot, bad dialogue, bad acting, idiotic di...         0
49997  I am a Catholic taught in parochial elementary...         0
49998  I'm going to have to disagree with the previou...         0
49999  No one expects the Star Trek movies to be high...         0

[50000 rows x 2 columns]


In [21]:
data["sentiment"].value_counts()

sentiment
1    25000
0    25000
Name: count, dtype: int64

In [22]:
# split data into training data and test data
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [23]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


In [24]:
#DATA PREPROCESSING

In [25]:
# Tokenize text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [26]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [27]:
print(X_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [28]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [29]:
print(Y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: object


In [30]:
#LSTM-LONG SHORT-TERM MEMORY

In [31]:
# build the model

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

C:\Users\nehaa\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [32]:
model.build(input_shape=(None, 200))

In [33]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 200, 128)            │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

In [34]:
# compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [35]:
#TRAINING THE MODEL

In [36]:
import numpy as np

In [37]:
Y_train = np.array(Y_train, dtype=np.float32)
Y_test = np.array(Y_test, dtype=np.float32)

X_train = np.array(X_train)
X_test = np.array(X_test)

In [38]:
model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 100s 196ms/step - accuracy: 0.7198 - loss: 0.5296 - val_accuracy: 0.7146 - val_loss: 0.5463
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 98s 196ms/step - accuracy: 0.8369 - loss: 0.3806 - val_accuracy: 0.8491 - val_loss: 0.3525
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 100s 199ms/step - accuracy: 0.8667 - loss: 0.3231 - val_accuracy: 0.8620 - val_loss: 0.3404
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 98s 195ms/step - accuracy: 0.8720 - loss: 0.3186 - val_accuracy: 0.8611 - val_loss: 0.3401
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 98s 197ms/step - accuracy: 0.8917 - loss: 0.2723 - val_accuracy: 0.8609 - val_loss: 0.3350


In [54]:
#MODEL EVALUATION
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 12s 38ms/step - accuracy: 0.8668 - loss: 0.3189
Test Loss: 0.3185097873210907
Test Accuracy: 0.8682000041007996


In [55]:
#BUILDING A PREDICTIVE SYSTEM

In [56]:
def predict_sentiment(review):
  # tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [57]:
# example usage
new_review = "This movie was fantastic. I loved it."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
The sentiment of the review is: positive


In [58]:
# example usage
new_review = "This movie was not that good"
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
The sentiment of the review is: positive


In [59]:
# example usage
new_review = "This movie was ok but not that good."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
The sentiment of the review is: positive


In [63]:
print(f"The sentiment of the review is: {predict_sentiment(input())}")

 The movie was awful


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
The sentiment of the review is: negative


In [64]:
import ipywidgets as widgets
from IPython.display import display

In [65]:
# Creating text input widget
review_input = widgets.Text(
    value='',
    placeholder='Type your movie review here',
    description='Review:',
    disabled=False
)

In [66]:
# Creating button widget
button = widgets.Button(description="Predict Sentiment")

In [67]:
# Creating output widget to display the result
output = widgets.Output()

In [68]:
# Defining the function to be called when the button is clicked
def on_button_clicked(b):
    with output:
        output.clear_output()  # Clear previous output
        review = review_input.value
        sentiment = predict_sentiment(review)
        print(f"The sentiment of the review is: {sentiment}")

In [69]:
# Linking the button click event to the function
button.on_click(on_button_clicked)

In [70]:
# Displaying the widgets
display(review_input, button, output)

Text(value='', description='Review:', placeholder='Type your movie review here')

Button(description='Predict Sentiment', style=ButtonStyle())

Output()

In [71]:
import tkinter as tk
from tkinter import messagebox
from tkinter import scrolledtext

In [72]:
# Creating Tkinter root widget
root = tk.Tk()
root.title("Sentiment Analysis")

''

In [73]:
# Function to handle button click
def analyze_sentiment():
    review = review_entry.get("1.0", tk.END).strip()  # Get text from the text area
    if review:
        sentiment = predict_sentiment(review)
        messagebox.showinfo("Sentiment Analysis Result", f"The sentiment of the review is: {sentiment}")
    else:
        messagebox.showwarning("Error", "Please enter a review.")

In [74]:
# Adding widgets to the root window
label = tk.Label(root, text="Enter your movie review:")
label.pack(pady=10)

In [75]:
review_entry = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=40, height=10)
review_entry.pack(pady=10)

In [76]:
analyze_button = tk.Button(root, text="Analyze Sentiment", command=analyze_sentiment)
analyze_button.pack(pady=10)

In [ ]:
# Running the Tkinter event loop (main loop)
root.mainloop()

In [ ]:
pip install dash dash-bootstrap-components


In [7]:
import dash
from dash import dcc, html, Input, Output, State
import dash_bootstrap_components as dbc

In [8]:
# Initializing the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

In [9]:
# Defining the layout of the app
app.layout = dbc.Container(
    [
        html.H1("Movie Review Sentiment Analysis"),
        dbc.Textarea(id="review-input", placeholder="Enter your movie review here...", style={"width": "100%", "height": 200}),
        dbc.Button("Analyze Sentiment", id="analyze-button", color="primary", className="mt-3"),
        html.Div(id="sentiment-output", className="mt-3")
    ],
    className="p-5",
)

In [10]:
# Defining callback to update sentiment output
@app.callback(
    Output("sentiment-output", "children"),
    Input("analyze-button", "n_clicks"),
    State("review-input", "value")
)
def update_output(n_clicks, review):
    if n_clicks and review:
        sentiment = predict_sentiment(review)
        return html.Div(f"The sentiment of the review is: {sentiment}", className="alert alert-info")
    return ""

In [11]:
# Running the app
if __name__ == '__main__':
    app.run_server(debug=True)